In [49]:
import json 
from typing import Tuple

def avg_function_gas_usage(contract: str) -> Tuple[float, float]:
    with open(f"results/{contract}_results.json", "r") as f:
        lines = f.readlines()
        txs = map(lambda l: json.loads(l), lines)
        gas_usage = list(map(lambda x: x['gas'], txs))
        baseline = sum(gas_usage) / len(gas_usage)

    with open(f"results/{contract}_transpiled_results.json", "r") as f:
        lines = f.readlines()
        txs = map(lambda l: json.loads(l), lines)
        gas_usage = list(map(lambda x: x['gas'], txs))
        consol = sum(gas_usage) / len(gas_usage)

    return (baseline, consol)


In [50]:
def wrapped_erc20_avg(contract: str) -> Tuple[float, float]:
    return avg_function_gas_usage(f"ERC20_{contract}")

def wrapped_erc721_avg(contract: str) -> Tuple[float, float]:
    return avg_function_gas_usage(f"ERC721_{contract}")

In [53]:
cs = [
    "TETHER",
    "ZRX",
    "THETA",
    "INB",
    "HEDG",
    "DAI",
    "EKT",
    "XIN",
    "HOT",
    "SWIPE",
]
avg_gas = [wrapped_erc20_avg(c) for c in cs]
print(' & '.join([f'{b:.1f}' for b, _ in avg_gas]))
print(' & '.join([f'{c:.1f}' for _, c in avg_gas]))
print(' & '.join([f'{(c-b)/b*100:.2f}\%' for b, c in avg_gas]))

62426.0 & 51468.0 & 51540.0 & 53738.0 & 53941.0 & 53696.0 & 51911.0 & 51375.0 & 51525.0 & 55728.0
63382.0 & 51468.0 & 51895.0 & 53735.0 & 54160.0 & 53960.0 & 52258.0 & 51727.0 & 51566.0 & 55952.0
1.53\% & 0.00\% & 0.69\% & -0.01\% & 0.41\% & 0.49\% & 0.67\% & 0.69\% & 0.08\% & 0.40\%


In [16]:
cs = [
    "MCHH",
    "CC",
    "CLV",
    "LAND",
    "CARDS",
    "KB",
    "TRINKET",
    "PACKS",
    "BKC",
    "EGG",
]
avg_gas = [wrapped_erc721_avg(c) for c in cs]
print(' & '.join([f'{b:.1f}' for b, _ in avg_gas]))
print(' & '.join([f'{c:.1f}' for _, c in avg_gas]))
print(' & '.join([f'{(c-b)/b*100:.2f}\%' for b, c in avg_gas]))

138033.5 & 133651.0 & 150692.0 & 134749.5 & 133786.5 & 133764.5 & 133497.0 & 156070.5 & 143674.0 & 134251.5
138576.0 & 134188.0 & 151439.5 & 135281.0 & 134174.5 & 134155.5 & 134017.0 & 156458.5 & 144013.0 & 134460.0
0.39\% & 0.40\% & 0.50\% & 0.39\% & 0.29\% & 0.29\% & 0.39\% & 0.25\% & 0.24\% & 0.16\%


In [18]:
gas = avg_function_gas_usage("ERC1202_VOTE")
print(gas)
print(f"{(gas[1] - gas[0])/gas[0]*100:.2f}\%")

(118292.5, 118410.5)
0.10\%


In [52]:
# BEC
with open("results/ERC20_BEC.sol_gas.json", "r") as f:
    content = json.load(f)
    gas_usage = list(filter(lambda x: x is not None, content))
    baseline = (sum(gas_usage) / len(gas_usage))

with open("results/ERC20_BEC_transpiled.sol_gas.json", "r") as f:
    content = json.load(f)
    gas_usage = list(filter(lambda x: x is not None, content))
    consol = (sum(gas_usage) / len(gas_usage))
print(baseline, consol)
print(f"{(consol-baseline)/baseline*100:.2f}\%")

114725.76963971407 115554.46624040614
0.72\%


In [20]:
# DOZ
with open("results/ERC721_DOZ.sol_gas.json", "r") as f:
    content = json.load(f)
    gas_usage = list(filter(lambda x: x is not None, content))
    baseline = (sum(gas_usage) / len(gas_usage))

with open("results/ERC721_DOZ_transpiled.sol_gas.json", "r") as f:
    content = json.load(f)
    gas_usage = list(filter(lambda x: x is not None, content))
    consol = (sum(gas_usage) / len(gas_usage))
print(baseline, consol)
print(f"{(consol-baseline)/baseline*100:.2f}\%")

255037.81187963716 255557.57195141056
0.20\%


# Decoupling Measurement    

In [30]:
import subprocess
from typing import Tuple

def calculate_diff(contract: str) -> Tuple[int, int]:
    folder = "results"
    try:
        subprocess.check_output(['diff', f'{folder}/{contract}.sol', f'{folder}/{contract}_spec.sol'])
        return 0, 0
    except subprocess.CalledProcessError as e:
        output = e.output

        add = 0
        delete = 0
        for line in output.splitlines():
            if line.startswith(b'<'):
                line = line[1:].strip()
                if len(line) > 0:
                    delete += 1
            elif line.startswith(b'>') and "consol" in str(line):
                line = line[1:].strip()
                if len(line) > 0:
                    add += 1
        return add, delete

In [39]:
cs = [
    "ERC20_BEC",
    "ERC20_TETHER",
    "ERC20_ZRX",
    "ERC20_THETA",
    "ERC20_INB",
    "ERC20_HEDG",
    "ERC20_DAI",
    "ERC20_EKT",
    "ERC20_XIN",
    "ERC20_HOT",
    "ERC20_SWIPE",
    "ERC1202_VOTE",
    "ERC721_DOZ",
    "ERC721_MCHH",
    "ERC721_CC",
    "ERC721_CLV",
    "ERC721_LAND",
    "ERC721_CARDS",
    "ERC721_KB",
    "ERC721_TRINKET",
    "ERC721_PACKS",
    "ERC721_BKC",
    "ERC721_EGG",
]
result = {}
for c in cs:
    result[c] = calculate_diff(c)

line1 = []
line2 = []
line3 = []
line4 = []
for c, (add, delete) in result.items():
    line1.append(c)
    line2.append(str(add).rjust(len(c), ' '))
    line3.append(str(delete).rjust(len(c), ' '))
    if add == 0:
        line4.append(f"-".rjust(len(c), ' '))
    else:
        line4.append(f"{delete/add:.2f}".rjust(len(c), ' '))
print(result)
print(" & ".join(line1))
print(" & ".join(line2))
print(" & ".join(line3))
print(" & ".join(line4))


{'ERC20_BEC': (7, 11), 'ERC20_TETHER': (4, 4), 'ERC20_ZRX': (0, 0), 'ERC20_THETA': (3, 3), 'ERC20_INB': (3, 5), 'ERC20_HEDG': (2, 3), 'ERC20_DAI': (2, 2), 'ERC20_EKT': (0, 2), 'ERC20_XIN': (3, 3), 'ERC20_HOT': (3, 5), 'ERC20_SWIPE': (2, 2), 'ERC1202_VOTE': (2, 3), 'ERC721_DOZ': (24, 29), 'ERC721_MCHH': (7, 9), 'ERC721_CC': (7, 8), 'ERC721_CLV': (7, 8), 'ERC721_LAND': (7, 9), 'ERC721_CARDS': (5, 8), 'ERC721_KB': (5, 7), 'ERC721_TRINKET': (7, 8), 'ERC721_PACKS': (5, 7), 'ERC721_BKC': (5, 8), 'ERC721_EGG': (7, 11)}
ERC20_BEC & ERC20_TETHER & ERC20_ZRX & ERC20_THETA & ERC20_INB & ERC20_HEDG & ERC20_DAI & ERC20_EKT & ERC20_XIN & ERC20_HOT & ERC20_SWIPE & ERC1202_VOTE & ERC721_DOZ & ERC721_MCHH & ERC721_CC & ERC721_CLV & ERC721_LAND & ERC721_CARDS & ERC721_KB & ERC721_TRINKET & ERC721_PACKS & ERC721_BKC & ERC721_EGG
        7 &            4 &         0 &           3 &         3 &          2 &         2 &         0 &         3 &         3 &           2 &            2 &         24 &           

In [48]:
cs = [
    "ERC20_BEC",
    "ERC20_TETHER",
    "ERC20_ZRX",
    "ERC20_THETA",
    "ERC20_INB",
    "ERC20_HEDG",
    "ERC20_DAI",
    "ERC20_EKT",
    "ERC20_XIN",
    "ERC20_HOT",
    "ERC20_SWIPE",
    "ERC1202_VOTE",
    "ERC721_DOZ",
    "ERC721_MCHH",
    "ERC721_CC",
    "ERC721_CLV",
    "ERC721_LAND",
    "ERC721_CARDS",
    "ERC721_KB",
    "ERC721_TRINKET",
    "ERC721_PACKS",
    "ERC721_BKC",
    "ERC721_EGG",
]

from typing import Tuple, List
import re

def calculate_function_logic_extraction_ratio(contract: str) -> List[Tuple[int, int]]:
    file = f"results/{contract}_spec.sol"
    with open(file, "r") as f:
        lines = f.readlines()
    fns = []
    for line in lines:
        match = re.search("@custom:consol-diff\s+(\d+)/(\d+)", line)    
        if match is None:
            continue
        fns.append((int(match.group(1)), int(match.group(2))))
    return fns

result = {}
for c in cs:
    result[c] = calculate_function_logic_extraction_ratio(c)
    
line1 = []
line2 = []
for c in result.keys():
    line1.append(c)
    if len(result[c]) == 0:
        line2.append("-".rjust(len(c), ' '))
        continue
    avg = sum([a/b for a, b in result[c]]) / len(result[c])
    line2.append(f"{avg*100:.2f}\%".rjust(len(c), ' '))
print(result)
print(" & ".join(line1))
print(" & ".join(line2))


{'ERC20_BEC': [(1, 3), (1, 2), (1, 3), (3, 7), (4, 9), (1, 3), (4, 11)], 'ERC20_TETHER': [(1, 5), (1, 2), (1, 3), (1, 5)], 'ERC20_ZRX': [], 'ERC20_THETA': [(1, 2), (1, 3), (2, 6)], 'ERC20_INB': [(1, 2), (1, 3), (2, 6)], 'ERC20_HEDG': [(1, 2), (1, 2)], 'ERC20_DAI': [(1, 2), (1, 2)], 'ERC20_EKT': [(1, 2), (1, 2), (3, 10), (2, 6)], 'ERC20_XIN': [(1, 2), (1, 3), (1, 5)], 'ERC20_HOT': [(1, 2), (1, 2), (2, 6)], 'ERC20_SWIPE': [(1, 2), (1, 2)], 'ERC1202_VOTE': [(2, 8), (1, 6)], 'ERC721_DOZ': [(1, 5), (1, 3), (1, 3), (1, 3), (1, 2), (1, 2), (1, 2), (1, 3), (2, 5), (1, 2), (1, 3), (1, 2), (2, 5), (1, 5), (2, 6), (1, 2), (1, 2), (1, 2), (1, 2), (2, 3), (2, 3), (1, 2), (1, 2), (1, 3)], 'ERC721_MCHH': [(1, 3), (1, 3)], 'ERC721_CC': [(1, 2), (1, 3), (2, 6), (1, 3), (1, 3), (1, 3), (1, 3)], 'ERC721_CLV': [(1, 2), (1, 3)], 'ERC721_LAND': [(1, 3), (1, 3), (2, 6), (1, 3), (1, 3), (1, 3), (1, 3)], 'ERC721_CARDS': [(1, 3), (1, 3), (2, 5), (2, 7)], 'ERC721_KB': [(1, 3), (1, 3), (1, 2), (2, 5), (2, 7)], 'E